In [1]:
# DATA2001 Week 10 Tutorial
# Material last updated: 2 May 2023
# Note: this notebook was designed with the Roboto Condensed font, which can be installed here: https://www.1001fonts.com/roboto-condensed-font.html

from IPython.display import HTML
HTML('''
    <style> body {font-family: "Roboto Condensed Light", "Roboto Condensed";} h2 {padding: 10px 12px; background-color: #E64626; position: static; color: #ffffff; font-size: 40px;} .text_cell_render p { font-size: 15px; } .text_cell_render h1 { font-size: 30px; } h1 {padding: 10px 12px; background-color: #E64626; color: #ffffff; font-size: 40px;} .text_cell_render h3 { padding: 10px 12px; background-color: #0148A4; position: static; color: #ffffff; font-size: 20px;} h4:before{ 
    content: "@"; font-family:"Wingdings"; font-style:regular; margin-right: 4px;} .text_cell_render h4 {padding: 8px; font-family: "Roboto Condensed Light"; position: static; font-style: italic; background-color: #FFB800; color: #ffffff; font-size: 18px; text-align: center; border-radius: 5px;}input[type=submit] {background-color: #E64626; border: solid; border-color: #734036; color: white; padding: 8px 16px; text-decoration: none; margin: 4px 2px; cursor: pointer; border-radius: 20px;}</style>
''')

# Week 10 - Time Series Data

This week's tutorial focusses on **time series data** - a collection of observations obtained through repeated measurements over time. We'll be focussing on a relevant real-world dataset, loading into the database, before exploring how the same information can be represented and queries in two separate ways (point based vs sequence based representations).

## 1. Data Loading

Our dataset for today contains financial trading information - the historical price range of Bitcoin in hourly windows. This has been downloaded from a [CryptoDataDownload](https://www.cryptodatadownload.com/data/gemini) website and is based on the Gemini exchange.

A more in-depth dataset extending back to 2018 is available on Canvas as "BitcoinPrices" for those interested in further analysis, but we'll be leveraging the "**BitcoinPricesSubset**" spreadsheet, which only contains data from 2021 onwards, or from July of the earlier years. The code used to generate this subset from the more complete dataset is included as comments in the cell below:

In [ ]:
import pandas as pd

#btcfull = pd.read_csv('BitcoinPrices.csv')
#btc = btcfull.copy()
#btc['date'] = pd.to_datetime(btc['unix'], unit='ms')
#btc = btc.loc[(btc['date'].dt.year >= 2021) | (btc['date'].dt.month == 7)]
#btc.drop(columns=['date'], inplace=True)
#btc.to_csv('BitcoinPricesSubset.csv', index=False)

### 1.1 Ingestion

As always, let's first load in the dataset using Pandas (noting this expects the CSV file to be in the same location as this notebook).

In [ ]:
btc = pd.read_csv('BitcoinPricesSubset.csv')
btc.head()

While one would certainly expect a 'date' column to indicate the time period each row concerns, the dataset does not contain an explicit datetime field. Rather, this information is stored in [Unix time](https://en.wikipedia.org/wiki/Unix_time), i.e. the number of seconds or milliseconds that have passed since the 1st of January 1970. All other fields are also numeric.

In [ ]:
btc.dtypes

Given Unix time is an internationally recognised format, Pandas has the ability to conver this to a datetime field using the `.to_datetime()` function. In this case, the unix information represents the number of milliseconds since 1970, so we'll specify that information in the 'unit' argument.

While there, let's also round the *volume_usd* column to the nearest cent, and drop a couple less relevant columns. This leaves us with one row per hour, with details of the opening and closing price, highest and lowest price, and volume in $US for that time period.

In [ ]:
btc['date'] = pd.to_datetime(btc['unix'],unit='ms')
btc['volume_usd'] = round(btc['volume_usd'], 2)
btc.drop(columns=['unix', 'volume_btc'], inplace=True)
btc.head()

### 1.2 Loading

A few familiar functions... (remember to include your `Credentials.json` file again!)

In [ ]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json
import os

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

db, conn = pgconnect(credentials)

Let's store this information in a **Prices** schema, within which we'll call the table **Bitcoin**.

In [ ]:
conn.execute("""
create schema if not exists prices;
set search_path to prices;
drop table if exists bitcoin;
""")

Leveraging Pandas' `.to_sql()` function, we can upload the data to our Bitcoin table. Note this cell may take a minute to run (when servers are largely unused it takes just under 20 seconds, I expect this to increase with multiple concurrent users!)

In [ ]:
btc.to_sql("bitcoin", con=conn, if_exists='append', index=False)
query(conn, "select * from bitcoin limit 10")

## 2. Point Based Representation

A point-based representation involves **multiple rows with atomic data types**. This is a familiar format, given it is how the data has been provided to us - one row per hour, with simple summary statistics of that time frame.

### 2.1 Examples

Examples for this format are less crucial, given our familiarity with building simple queries. For a simple case though, the following query would find the number of hours in the dataset for which the **price varied by at least US$5000**.

In [ ]:
sql = """
select count(*)
from Bitcoin
where High-Low >= 5000
"""
result = query(conn, sql)
result

### 2.2 Tasks

Let's refresh our basic SQL skills by answering the following three questions:

**Task: a)** What is the lowest and highest close price that Bitcoin has reached so far in 2023?

In [ ]:
## TO DO

**Task: b)** What was the average midnight open price of Bitcoin over the last month (April 2023), to 2 decimal places?

*Note: the rounding process may require a conversion to the `numeric` datatype (see the [docs](https://www.postgresql.org/docs/8.1/functions-math.html))*

In [ ]:
## TO DO

**Task: c)** What was the average hourly volume (in $US) traded in July of each year?

In [ ]:
## TO DO

## 3. Sequence Based Representation

Now let's pivot to a new storage method for time series data. A sequenced-based representation involves grouping information into a **single row containing an array of time point data**. Take the following example based on a subset of our data:

Point-based:

| date | high | low |
| :--- | :--- | :--- |
| 11-Mar-2023 00:00:00 | 39441.61 | 39154.63 |
| 11-Mar-2023 01:00:00 | 39342.98 | 38600.00 |
| 11-Mar-2023 02:00:00 | 38757.44 | 38254.50 |

Sequence-based:

| date | high | low |
| :--- | :--- | :--- |
| 11-Mar-2023 | [39441.61, 39342.98, 38757.44] | [39154.63, 38600.00, 38254.50] |

### 3.1 Examples

Whenever we seek to create a `group by` query in SQL, all columns must be either in the grouping clause, or an aggregation (e.g. `max`, `avg`, etc). For example, if we wanted to group our data at a daily level rather than an hourly, we could return the average open price, and the total $US amount traded for each day, like below:

In [ ]:
sql = """
select date(Date), avg(Open), sum(Volume_USD)
from Bitcoin
group by date(Date)
order by Date
limit 3
"""
query(conn, sql)

Rather than produce a *single number* with each aggreggation metric like above, we can return a **complete array** of the relevant values, such as below, where the opening price of each individual hour for that day are bundled up as lists.

In [ ]:
sql = """
select date(Date), array_agg(Open order by Date)
from Bitcoin
group by date(Date)
order by Date
limit 3
"""
query(conn, sql)

#### How do we build queries with this information format?

To further delve into what's possible with a sequence based format, let's store this example as a subquery/CTE (Common Table Expression) using the `with` clause. This allows us to define a temporary table, here named **Bitcoin2**, from which we can query.

In [ ]:
sql = """
with Bitcoin2 as (
  select date(Date), array_agg(Open order by Date) as openPrices
  from Bitcoin
  group by date(Date)
  order by Date
  limit 3
)
select * from Bitcoin2
"""
query(conn, sql)

When information is stored this way, we can unravel it using the `unnest()` function:

In [ ]:
sql = """
with Bitcoin2 as (
  select date(Date), array_agg(Open order by Date) as openPrices
  from Bitcoin
  group by date(Date)
  order by Date
  limit 3
)
select Date, unnest(openPrices) as Open
from Bitcoin2
"""
query(conn, sql)

Note though that this hasn't fully retained the date information - the column now only contains the date, not the hour. We can ensure the hour is represented with the `generate_series()` function, which effectively includes an index column accordingly.

In [ ]:
sql = """
with Bitcoin2 as (
  select date(Date), array_agg(Open order by Date) as openPrices
  from Bitcoin
  group by date(Date)
  order by Date
  limit 3
)
select Date, generate_series(1, 24)-1 as hour, unnest(openPrices) as Open
from Bitcoin2
"""
query(conn, sql)

We can also just extract individual elements from the array, such as the query below, which extracts the first and last open price.

*(Note: this is unfortunately indexed from 1, not 0...)*

In [ ]:
sql = """
with Bitcoin2 as (
  select date(Date), array_agg(Open order by Date) as openPrices
  from Bitcoin
  group by date(Date)
  order by Date
  limit 3
)
select Date, openPrices[1] as FirstOpen, openPrices[24] as LastOpen
from Bitcoin2
"""
query(conn, sql)

The `cardinality()` function also allows us to determine how long a given array is:

In [ ]:
sql = """
with Bitcoin2 as (
  select date(Date), array_agg(Open order by Date) as openPrices
  from Bitcoin
  group by date(Date)
  order by Date
  limit 3
)
select Date, cardinality(openPrices)
from Bitcoin2
"""
query(conn, sql)

#### How can we more permanently store this representation?

Rather than rely on a subquery continuously like above, let's create a **Bitcoin2** table to store the Bitcoin price data this way. This can be defined using arrays of a set data type, e.g. `FLOAT[]` indicates an array of floats.

From there, we can insert data based on a simple query that aggregates our information into arrays. Note the importance of a sort - here by Date to ensure it reflects the true hour order.

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS Bitcoin2;
CREATE TABLE IF NOT EXISTS Bitcoin2 (
    Date DATE,
    Open NUMERIC[],
    High NUMERIC[],
    Low NUMERIC[],
    Close NUMERIC[],
    Volume_USD NUMERIC[]
);

INSERT INTO Bitcoin2 (
    SELECT
        date(Date),
        array_agg(Open ORDER BY Date),
        array_agg(High ORDER BY Date),
        array_agg(Low ORDER BY Date),
        array_agg(Close ORDER BY Date),
        array_agg(Volume_USD ORDER BY Date)
    FROM Bitcoin
    GROUP BY date(Date)
);
""")

Investigating the resulting format, we can see a row for each date, with a 24-length array for just about all dates containing the hour-by-hour breakdown of each metric (the 2nd of May has only one value, since there's only information for midnight on that date).

In [ ]:
query(conn, "select * from Bitcoin2 order by Date desc limit 3")

### 3.2 Tasks

From there, let's try and answer the *same* questions from before using this new format. Answer the following questions **using the Bitcoin2 sequence-based representation**, and any relevant functions such as those demonstrated above. For further reading, feel free to consult the [docs on PostgreSQL array functions](https://www.postgresql.org/docs/current/functions-array.html).

**Task: a)** What is the lowest and highest close price that Bitcoin reached in 2023?

In [ ]:
## TO DO

**Task: b)** What was the average midnight open price of Bitcoin over the last month (April 2023), to 2 decimal places?

In [ ]:
## TO DO

**Task: c)** What was the average hourly volume (in $US) traded in December of each year?

In [ ]:
## TO DO

## 4. Inserting New Data

When **new information** is available, how can this be added to our tables, in either representation?

### 4.1 Point Based

The data was provided up until the end of April 2023. Let's test adding two test rows of data for demonstration purposes:

| date | open | high | low | close | volume_usd |
| :--- | :--- | :--- | :--- | :--- | :--- |
| 01-May-2023 00:00:00 | 29200.00 | 29321.54 | 29123.45 | 29222.22 | 234567.89 |
| 01-May-2023 01:00:00 | 29333.33 | 29929.29 | 29020.20 | 29200.02 | 345678.90 |

For our simple point-based representation, this involves `insert into` statements like so:

In [ ]:
conn.execute("""
insert into Bitcoin values(29200.00, 29321.54, 29123.45, 29222.22, 234567.89, '01-May-2023 00:00:00');
insert into Bitcoin values(29333.33, 29929.29, 29020.20, 29200.02, 345678.90, '01-May-2023 01:00:00');
""")

Returning the five most recent hours confirms the additions were successful:

In [ ]:
query(conn, "select * from Bitcoin order by Date desc limit 5")

### 4.2 Sequence Based

Let's now seek to add the same new rows of information for our sequence based representation:

| date | open | high | low | close | volume_usd |
| :--- | :--- | :--- | :--- | :--- | :--- |
| 01-May-2023 00:00:00 | 29200.00 | 29321.54 | 29123.45 | 29222.22 | 234567.89 |
| 01-May-2023 01:00:00 | 29333.33 | 29929.29 | 29020.20 | 29200.02 | 345678.90 |

When adding information for the 1st of May, no current row exists. We can leverage the `insert into` clause, with our data explicitly stored in simple arrays, so that other values can be added into it later.

For demonstration purposes, we'll just add the first row there to start:

In [ ]:
conn.execute("""
insert into Bitcoin2 values(
    '2023-05-01',
    array[29200.00],
    array[29321.54],
    array[29123.45],
    array[29222.22],
    array[234567.89]
)
""")

Adding in that row for the second hour allows us to see how data can be added to an existing array, using the `array_append()` function within an `update` statement.

In [ ]:
conn.execute("""
update Bitcoin2
set open = array_append(open, 29333.33),
    high = array_append(high, 29929.29),
    low = array_append(low, 29020.20),
    close = array_append(close, 29200.02),
    volume_usd = array_append(volume_usd, 345678.90)
where date = '2023-05-01'
""")

Again confirming this is successful:

In [ ]:
query(conn, "select * from Bitcoin2 order by Date desc limit 3")

Since these were test examples, it's worth noting that a `delete` statement in SQL allows rows to be dropped according to a given filter. The code below will drop these test rows, as shown by then re-running the statement above.

In [ ]:
conn.execute("delete from Bitcoin2 where date >= '2023-05-01'")
query(conn, "select * from Bitcoin2 order by Date desc limit 3")

## 5. Visuals

As a final demonstration, let's briefly investigate some potential visualisations that can be built from querying time-series data.

### 5.1 Line Graph

Let's return to the base table, and return an aggregation of **all Open prices for July months** in our dataset.

In [ ]:
sql = """
select extract(year from Date) as year, array_agg(open order by Date)
from Bitcoin
where extract(month from Date) = 7
group by extract(year from Date)
"""
result = query(conn, sql)
result

From there, we can take this data and visualise each as a line graph for comparison.

*(Note the code line involving a `zip()` function isn't super neat, but necessary to achieve the format matplotlib requires - which is effectively a pivot of how the information is returned. Open to alternative ways people find for this if you're interested!)*

In [ ]:
import matplotlib.pyplot as plt
plt.plot(list(zip(*result['array_agg'].tolist())))
plt.legend(result['year'], loc='lower right', title='Year')
plt.title("BTC Price in July", fontdict={'fontsize': 14}, pad=10)  # setting a title
plt.xlabel("Hours Since Start of Month", labelpad=10)  # setting an x-axis
plt.ylabel("Price ($US)", labelpad=10)  # setting a y-axis
plt.ylim(0, )
plt.show()

### 5.2 Histogram

Rather than visualising the changes over the course of a month, we can create visuals to find trends during the day. The below query finds **for each hour of the day, how often the trading price range has exceeded $200 so far this year**.

In [ ]:
sql = """
select n, count(*)
from (
  select date, generate_series(1, 24) n, high, low
  from Bitcoin2
) t
where high[n]-low[n] >= 200 and extract(year from date) = 2023
group by n
order by n
"""
result2 = query(conn, sql)
result2

The resulting graph contains some potentially interesting trends for the hours of the day most commonly involving higher trading swings. What could possibly be surmised from this graph?

In [ ]:
plt.bar(result2['n'], result2['count'])

#### Don't forget to close your connection!

That's all for today's tutorial - enjoy your week and remember the group assignment is due in two weeks! :)

In [ ]:
conn.close()
db.dispose()